In [1]:
import praw
import time
import pandas as pd
import csv
from praw.models import Submission
import os
from tenacity import retry, stop_after_attempt, wait_exponential
from tqdm import tqdm 

In [2]:
# Define necessary names
SUBREDDIT_NAME = "UBC"       
MAX_POSTS = 10000                       
BATCH_SIZE = 100                       
CHECKPOINT_FILE = "checkpoint.txt"      
POSTS_CSV = "r_ubc_posts.csv"                 
COMMENTS_CSV = "r_ubc_comments.csv"           
REQUEST_DELAY = 1

In [3]:
# Load reddit api throgh praw
reddit = praw.Reddit(
    client_id="qg6qmpLE5NHZ98F71GBiiA",
    client_secret="5flQrSnLQjGwIoLQIrV_YFluxej9GQ",
    user_agent="script:my_script:v1.0 (by /u/NathanPalaiologos)"
)



In [4]:
# Set csv file

pd.DataFrame(columns=[
    "post_id", "title", "author", "content",
    "score", "num_comments", "created_utc", "url"
]).to_csv(POSTS_CSV, index=False)


pd.DataFrame(columns=[
    "post_id", "comment_id", "author", "body", 
    "score", "created_utc"
]).to_csv(COMMENTS_CSV, index=False)

In [5]:
posts_file = open("r_ubc_posts.csv", "w", newline="", encoding="utf-8")
comments_file = open("r_ubc_comments.csv", "w", newline="", encoding="utf-8")

In [6]:
posts_writer = csv.writer(posts_file)
comments_writer = csv.writer(comments_file)
posts_writer.writerow(["id", "title", "author", "selftext", "score", "num_comments", "created_utc", "url"])
comments_writer.writerow(["post_id", "comment_id", "author", "body", "score", "created_utc"])


50

In [7]:
subreddit_name = "UBC" 
subreddit = reddit.subreddit(subreddit_name)

In [8]:
# Process comments
def process_comment(comment, post_id, csv_writer):
    csv_writer.writerow([
        post_id,
        comment.id,
        comment.author,
        comment.body,
        comment.score,
        comment.created_utc
    ])
    for reply in comment.replies:
        process_comment(reply, post_id, csv_writer)

# collect data
try:
    for submission in subreddit.new(limit=10000):
        print(f"Posts: {submission.title}")
        posts_writer.writerow([
            submission.id,
            submission.title,
            submission.author,
            submission.selftext,
            submission.score,
            submission.num_comments,
            submission.created_utc,
            submission.url
        ])

        # comments collection
        submission.comments.replace_more(limit=10)
        for comment in submission.comments.list():
            process_comment(comment, submission.id, comments_writer)

        time.sleep(1)  # Set delay to avoid blocking
finally:
    posts_file.close()
    comments_file.close()

Posts: pottery workshop tonight +1 needed
Posts: Year around housing contract
Posts: Is there a way to get my transcript for free
Posts: Does the STAT dept offer graduating class composites?
Posts: Can You Choose a Specific Building in Totem Park?
Posts: Can you take courses as an elective then use them as a minor afterwards
Posts: Has anyone received a response for undergraduate summer math research applications this year?
Posts: Mid Year Transfer
Posts: Hydrogen becomes a superfluid at nanoscale, confirming 50-year-old prediction
Posts: Is there a surefired way to get a work learn position?
Posts: Help: where to get Grad dresses
Posts: Free Conference Opportunity in Calgary!
Posts: BCampus resources
Posts: UBC Accelerated BSN program inquires
Posts: Do i need a tutor for cpsc210
Posts: tuition payment
Posts: YRH housing offers?
Posts: $24k stipend in Vancouver
Posts: midterm today BOOO
Posts: MY SLEEP SCHEDULE IS FUCKED
Posts: Grad Application Issue?
Posts: Do profs/instructors get i